satisfaction_level - 
Level of satisfaction (0-1) - 
Numeric  
last_evaluation - 
1 - 
Numeric  
number_project - 
Number of projects completed while at work - 
Numeric  
average_montly_hours - 
Average monthly hours at workplace - 
Numeric  
time_spend_company - 
Number of years spent in the company - 
Numeric  
Work_accident - 
Whether the employee had a workplace accident - 
Numeric  
left - 
Whether the employee left the workplace or not (1 or 0) Factor - 
Numeric   
promotion_last_5years - 
Whether the employee was promoted in the last five years -
Numeric  
sales -
Department in which they work for -
String  
salary - 
Relative level of salary (high) -
String

Dataset:
https://www.kaggle.com/ludobenistant/hr-analytics

In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [2]:
file = 'C:\\Users\\Abe\\Data Science Bootcamp\\Unit 3\\Advanced Regression\\Challenge\\HR_comma_sep.csv'
df = pd.read_csv(file)

# Getting Memory Error with all the data.
# df.sample(frac=.8,replace=True)

In [3]:
df_objects = df.select_dtypes(include=['object'])
for col in df_objects.columns:
    print(col)
    print(df[col].unique())

sales
['sales' 'accounting' 'hr' 'technical' 'support' 'management' 'IT'
 'product_mng' 'marketing' 'RandD']
salary
['low' 'medium' 'high']


In [4]:
Y = df['left']
X = df.drop(['left'],axis=1)
X = pd.get_dummies(X)

In [5]:
#Vanilla Logistic Regression
#Use l2 regression and an extremely high value for C to (almost) remove the penalty
v_regr = linear_model.LogisticRegression(C=1e9,penalty='l2')

v_regr.fit(X,Y)
v_score = cross_val_score(v_regr,X,Y,cv=10)
print(v_score)
print(v_score.mean())

[ 0.80746169  0.79066667  0.79466667  0.78666667  0.806       0.806
  0.79333333  0.788       0.74716478  0.73649099]
0.785645079605


In [6]:
#Ridge Regression
r_regr = linear_model.LogisticRegression(C=.5,penalty='l2')

r_regr.fit(X,Y)
r_score = cross_val_score(r_regr,X,Y,cv=10)
print(r_score)
print(r_score.mean())

[ 0.80746169  0.78866667  0.79333333  0.78466667  0.804       0.804
  0.79133333  0.78933333  0.74382922  0.73315544]
0.783977968198


In [7]:
l_regr = linear_model.LogisticRegression(C=.5,penalty='l1')

l_regr.fit(X,Y)
l_score = cross_val_score(l_regr,X,Y,cv=10)
print(l_score)
print(l_score.mean())

[ 0.80746169  0.79        0.79466667  0.78533333  0.804       0.80533333
  0.796       0.78933333  0.74649767  0.73649099]
0.785511701798


In [9]:
params = pd.DataFrame()

params['Features'] = X.columns
params['Vanilla'] = v_regr.coef_[0]
params['Ridge'] = r_regr.coef_[0]
params['Lasso'] = l_regr.coef_[0]
params

,Features,Vanilla,Ridge,Lasso
0,satisfaction_level,-4.117259,-4.037027,-4.108213
1,last_evaluation,0.769927,0.699809,0.667461
2,number_project,-0.314953,-0.307126,-0.309603
3,average_montly_hours,0.004414,0.004400,0.004433
4,time_spend_company,0.266324,0.264342,0.264730
5,Work_accident,-1.525651,-1.498874,-1.511743
6,promotion_last_5years,-1.211156,-1.129725,-1.306843
7,sales_IT,-0.106452,-0.104998,-0.146147
8,sales_RandD,-0.506033,-0.494244,-0.537139
9,sales_accounting,0.071539,0.070394,0.003339


# Notes
Out of the 3 models I would prefer the vanilla version. The data set has a manageable amount of features. Vanilla also gives the best R^2 while explaining each feature. The lasso model is very close in terms of R^2, but we lose out on the explanations for sales_marketing and salary_medium, which might be important to a stakeholder who has to make decisions from this model. 

As an overall model, regression is nice for its ability to give an explanation of the effect of an individual feature. It weaknesses are apparent in its strict assumptions which limit its applications.